# Imports, Functions & Testing

In [1]:
import pandas as pd
import numpy as np
import requests
import re
import pickle
from bs4 import BeautifulSoup

In [2]:
def wiki_senate_scraper():
    import pandas as pd
    import numpy as np
    import requests
    from bs4 import BeautifulSoup
    
    ## Starting page + Q.C. of response
    start_url = 'https://en.wikipedia.org/wiki/List_of_United_States_Senate_elections'
    start_resp = requests.get(start_url)
    print(f'Starting Response: {start_resp}')
    
    ## Creating soup + pulling all links for senate elections after 17th amendment
    start_soup = BeautifulSoup(start_resp.text, 'html.parser')
    start_links = start_soup.findAll('a')
    start_sen_links = start_links[78:134] ## Previously located
    
    ## Base of url for all senate election pages
    base_url = 'https://en.wikipedia.org'
    
    ## Loop same process for all links + storage
    yr_dfs = {}
    yr_tocs = {}
    count = 0
    for link in start_sen_links:

        ## Q.C during execute
        count += 1
        if count in [25, 50]:
            print('Checkpoint! (25 loops)')
        
        ## Collecting strings for use
        end_url = link.get('href')
        year = link.get_text()
        full_url = base_url + end_url
        
        ## Making soup + collecting all tables
        link_resp = requests.get(full_url)
        link_soup = BeautifulSoup(link_resp.text, 'html.parser')
        link_tables = link_soup.findAll('table', attrs={'class': ['wikitable', 'infobox vevent', 'infobox']})
        
        ## Collecting list of states with elections in each year
        link_toc = link_soup.find('div', attrs={'id':'toc'})
        link_toc = link_toc.findAll('a', href=is_state)
        toc_list = [tag.get('href').replace('#', '') for tag in link_toc]
        
        ## Converting to dataframe + storage
        elect_df = pd.read_html(str(link_tables))
        yr_dfs[year] = elect_df
        yr_tocs[year] = toc_list
    
    print(f'Total pages scraped: {count}')
    return yr_dfs, yr_tocs

## https://www.crummy.com/software/BeautifulSoup/bs4/doc/#kinds-of-filters

In [3]:
def is_state(href):
    states_list = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 
                   'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
                   'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
                   'Kentucky', 'Louisiana', 'Maine', 'Maryland',
                   'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
                   'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New_Hampshire',
                   'New_Jersey', 'New_Mexico', 'New_York', 'North_Carolina',
                   'North_Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
                   'Rhode_Island', 'South_Carolina', 'South_Dakota',
                   'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia',
                   'Washington', 'West_Virginia', 'Wisconsin', 'Wyoming']
    for state in states_list:
        if state in href:
            return href

In [4]:
def election_collector(dict_tables, dict_lists, yr_end='2020'):
    
    ## Requirement of proper dictionaries
    if dict_tables.keys() != dict_lists.keys():
        print('**WARNING**')
        print('Keys do not match in dictionaries passed. Adjust and try again.')
        return '***'*10
    
    ## Containers for results
    coll_elects = {}
    
    ## List creation for looping through dicts
    yr_list = list(dict_lists.keys())
    
    ## Looping + storage
    for year in yr_list:
        if year == yr_end:
            break
        print('***'*10)
        print(f"Collecting {year}'s elections...")
        yr_tables = dict_tables[year]
        yr_toc = dict_lists[year]
        
        ## Creating containers to further separate data
        yr_sum_ldrs = yr_tables[2]
        most_tables = yr_tables[3:]
        yr_summary = []
        yr_states = []
        count = 0
        
        for i, df in enumerate(most_tables):
            if count < 1:
                if year == '2018':
                    count += 1
                    yr_summary.append(yr_tables[10])
                    yr_summary.append(yr_tables[11])
                    continue
                elif df.shape[1] is 6 and most_tables[i+1].shape[1] is 6:
                    count += 1
                    yr_summary.append(most_tables[i])
                    yr_summary.append(most_tables[i+1])
                    continue
                elif df.shape[1] is 6:
                    count += 1
                    yr_summary.append(most_tables[i])
            
            if df.shape[1] in [4,5,6] and df.shape[0] < 15:
                yr_states.append(most_tables[i])
        
        coll_elects[year] = [yr_sum_ldrs, yr_summary, yr_states]
        
    print('\n')
    print('---'*20)
    disp_yrs = list(coll_elects.keys())
    print(f'Collection Complete: Data is for {disp_yrs[0]} - {disp_yrs[-1]}')
    print('---'*20)
    return coll_elects

In [79]:
def yr_summary_collector(election_collection):
    
    ## Container for results
    yr_summaries = {}
    
    ## Extraction as a list for iteration
    for year in election_collection:
        elect_list = [election_collection[year]]
        
        ## Looping through year's collection
        for coll in elect_list:
            ## Container for mid-results
            holder = []
            ## Looping through each table in 'year summary' group
            for table in coll[1]:
                ## Variable to filter unwanted tables out
                has_elected = False
                ## Looping through each row in 'results' column (idx 4)
                for u in table.iloc[:,4]:
                ## Checking for str to 'flag' table as collectable
                    if isinstance(u, str) and 'elected' in u:
                        has_elected = True
                if has_elected == True:
                    holder.append(table)
            ## Check if multiple tables in 'year summary' group
            if len(holder) > 1:
                is_match = holder[0].iloc[1,:] == holder[1].iloc[1,:]
                ## Check if tables have matching headers for special 
                ## election years + storage
                if is_match.sum() in [2,3]:
                    fin_tab = holder[1].append(holder[0], ignore_index=True)
                    yr_summaries[year] = fin_tab
            ## Storage of non-special election years
            else:
                try:
                    yr_summaries[year] = holder.pop()
                except:
                    continue
        
    return yr_summaries

In [6]:
def ref_tabler(summary_df, mod_df=True):
    ## Containers for mid-results
    st_list = []
    sntr_list = []
    
    ## Looping through states' names
    for state in summary_df['State']:
        ## Check/replace spaces with '_' + storage
        if ' ' in state:
            fmt_state = state.replace(' ', '_')
            st_list.append(fmt_state)
        else:
            st_list.append(state)
    ## Looping + storing senator names
    for name in summary_df['Senator']:
        sntr_list.append(name)

    ## Dataframe with Sentors as index + states as values
    state_ref_df = pd.DataFrame({'State_id':st_list, 'Incumbent':sntr_list})
    #state_ref_df.columns = ['State_id']
    
    ## Modifying list of states to that of the ref table OR not + return
    if mod_df:
        summary_df['State'] = st_list
        return summary_df, state_ref_df
    else:
        return state_ref_df

In [37]:
def master_tabler(yr_summary_dict):

    holder = []

    for year in yr_summary_dict:
        holder.append(yr_summary_dict[year][1])

    name_lookup_df = pd.concat(holder, ignore_index=True)
    
    return name_lookup_df

In [7]:
def yr_sum_formatter(yr_sum_dict, ref_table=True):
    ## Container for results + column helper
    res_dict = {}
    df_cols = ['State', 'Senator', 'Party', 'Electoral_History', 'Results', 'Candidates']
    
    ## Looping to each year's summary table
    for year in yr_sum_dict:
        ## Setting copy to modify + setting columns + rows to remove
        summary_df = yr_sum_dict[year].copy()
        summary_df.columns = df_cols
        drop_1 = summary_df.iloc[0]
        drop_2 = summary_df.iloc[1]
        
        ## Removing rows that match 'drop_' rows (former column info)
        for idx, row in summary_df.iterrows():
            if drop_1.equals(row):
                summary_df.drop(idx, inplace=True)
            elif drop_2.equals(row):
                summary_df.drop(idx, inplace=True)
        
        ## Resetting index to rangeIndex
        summary_df.reset_index(drop=True, inplace=True)
        
        ## Storing State-Senator lookup dataframe OR not
        if ref_table:
            summary_df, state_ref_df = ref_tabler(summary_df)
            state_ref_df['Year'] = year
            res_dict[year] = [summary_df, state_ref_df]    
        else:
            res_dict[year] = summary_df
        
    ## Return results
    return res_dict

In [8]:
def st_election_cleaner(dict_of_lists, exclude=[]):
    st_gens_cln = {}

    for year in dict_of_lists:
        if year in exclude:
            continue
        coll = dict_of_lists[year]
        holder = []
        
        for table in coll:
            if table.shape[1] in [5,6]:
                try:
                    table_cln = st_election_formatter(table)
                    holder.append(table_cln)
                except KeyError as e:
                    print(f'Error!! {e}')
                    print(f'Year affected: {year}')
                    display(table)
        
        st_gens_cln[year] = holder
        
    return st_gens_cln

In [9]:
def st_election_collector(election_collection):
    
    ## Container for results
    st_elects = {}
    
    ## Extraction of each year's state elections
    for year in election_collection:
        elect_list = election_collection[year]
        st_elects[year] = elect_list[2]
        
    return st_elects

In [10]:
def st_election_formatter(df):
    df_cln = df.copy()
    
    cols = ['Drop']
    cols.extend(df_cln.iloc[0,:-1])
    df_cln.columns = cols
    df_cln.drop(0, inplace=True)
    
    for i, v, na in zip(df_cln.index, df_cln['Drop'], df_cln['Drop'].isna()):
        if not na:
            df_cln[v] = df_cln['Party'][i]
            
    df_cln.drop(columns='Drop', inplace=True)
    
    for col in df_cln:
        if df_cln[col].isna().sum() == df_cln.shape[0]:
            df_cln.drop(columns=col, inplace=True)
            
        elif df_cln[col].isna().sum() > 0:
            for i, v, na in zip(df_cln.index, df_cln[col], df_cln[col].isna()):
                if na:
                    df_cln.drop(i, inplace=True)
    
    return df_cln

# Initial Data

> [Link to Dataset](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/PEJ5QU)

In [ ]:
df = pd.read_csv('dataverse_files/1976-2018-senate.csv', encoding='latin-1')
df.head()

In [ ]:
df.info()

In [ ]:
df[(df['state_po'] == 'AZ') & (df['year'] == 1976)]

In [ ]:
# len(df[df['candidate'].isna()])

# df['writein'].value_counts()

# for col in df:
#     display(df[col].value_counts())

# pd.Series.value_counts()

# Additional Data

Current set of features includes:
* Year of election
* State (encoded)
* 

> [Link to Brookings](https://www.brookings.edu/multi-chapter-report/vital-statistics-on-congress/)

> [Link to BallotPedia](https://ballotpedia.org/Legislative_Branch)

> [Link to WikiPedia](https://en.wikipedia.org/wiki/List_of_United_States_Senate_elections)

> [Link to Wiki-Category (U.S. Senator)](https://commons.wikimedia.org/wiki/Category:Senators_of_the_United_States)

## Testing tables 'infobox vevent'

In [ ]:
# url3 = 'https://en.wikipedia.org/wiki/1978_United_States_Senate_elections'
# response3 = requests.get(url3)
# print(response3)

# soup3 = BeautifulSoup(response3.text, 'html.parser')
# wiki_tables3 = soup3.findAll('table', attrs={'class': ['wikitable', 'infobox vevent']})

# print(type(wiki_tables3))

# test_df3 = pd.read_html(str(wiki_tables3))

# tester = test_df3[0]

In [ ]:
# https://stackoverflow.com/questions/27156278/index-pandas-dataframe-by-column-numbers-when-column-names-are-integers
# tester.columns = ['a', 'b', 'c']
# display(tester.iloc[6:18,0:3])

# display(test_df3[2])

In [ ]:
# url = 'https://en.wikipedia.org/wiki/1976_United_States_Senate_elections'
# response = requests.get(url)
# print(response)

# soup = BeautifulSoup(response.text, 'html.parser')
# wiki_tables = soup.findAll('table', attrs={'class': 'wikitable'})

# type(wiki_tables)

In [ ]:
# test_df = pd.read_html(str(wiki_tables))
# display(test_df[0])

# len(test_df)

## Pulling state election tables

In [ ]:
# ## Pulling Arizona gen election results
# tester = test_df[5].copy()
# print('O.G. Copy:')
# print('---'*20)
# display(tester)

# ## Setting columns manually + Q.C.
# tester.columns = ['Null', 'Party', 'Candidate', 'Votes', '%of_Vote']
# print('\nNew Column Headers:')
# print('---'*20)
# display(tester)

# ## Dropping irrelevant information + Q.C.
# tester.drop(index=[0, 6, 8], inplace=True)
# print('\nDropping Rows:')
# print('---'*20)
# display(tester)

# ## Flip and place Turnout as its own feature + Q.C.
# for idx, v, k in zip(tester.index, tester.Null, tester.Null.isna()):
#     if not k:
#         #print(idx, v)
#         tester[v] = tester.Party[idx]

# ## Removing the row with turnout info/column with nulls + Q.C.
# try:
#     tester.drop(columns='Null', index=7, inplace=True)
#     print('\nFinal Version:')
#     print('---'*20)
#     display(tester)
# except:
#     print('Something went wrong!')

In [ ]:
# type(tester['Votes'][5]) # str

In [ ]:
# tester2 = gen_election_cleaner(test_df[6], verbose=2)

In [ ]:
# test_df[-1]

In [ ]:
# url2 = 'https://en.wikipedia.org/wiki/List_of_United_States_Senate_elections'
# response2 = requests.get(url2)
# print(response2)

In [ ]:
# url2 = 'https://en.wikipedia.org/wiki/2018_United_States_Senate_elections'
# response2 = requests.get(url2)
# print(response2)

In [ ]:
# soup2 = BeautifulSoup(response2.text, 'html.parser')
# wiki_toc = soup2.find('div', attrs={'id':'toc'})
# wiki_toc = wiki_toc.findAll('a', href=is_state)
# display(wiki_toc[0:5])
# jiji = wiki_toc[0:5]

# jiji = [tag.get('href').replace('#', '') for tag in jiji]
# jiji = jiji
# jiji

## Working

In [ ]:
# test = wiki_links[78:134]
# beeb = test[0].get_text()
# geeb = 'https://en.wikipedia.org'
# geeb + beeb

In [ ]:
# display(elect_yr_dfs.keys())
# display(elect_yr_tocs.keys())
# display(len(elect_yr_dfs) == len(elect_yr_tocs))
# elect_yr_dfs.keys() != elect_yr_tocs.keys()
# list(elect_yr_dfs.keys())

In [ ]:
# elect_yr_dfs['1918']

In [ ]:
# elect_yr_tocs['1918']

In [ ]:
# year = '1924'

# for i, df in enumerate(elect_yr_dfs[year]):
#     print(i, df.shape)

# display(elect_yr_dfs[year][10])
# elect_yr_dfs[year][10].shape[1] is 6

In [ ]:
# koko = {v:elect_yr_dfs[v] for v in ['1918', '1924', '1976', '2016', '2018']}
# lplp = {v:elect_yr_tocs[v] for v in ['1918', '1924', '1976', '2016', '2018']}

In [ ]:
# year = '2016'

# for v in ftft[year]:
#     print(len(v))
    
# ftft[year][1]

In [ ]:
# yuyu = test_res[-1].append(test_res[-2], ignore_index=True)
# yuyu

In [ ]:
# yuyu = test_res[-1].iloc[1,:] == test_res[-2].iloc[1,:]
# yuyu.sum()

In [ ]:
# len(test_res)

## Getting it together

In [36]:
# elect_yr_tables, elect_yr_tocs = wiki_senate_scraper()

# election_collection_raw = election_collector(elect_yr_tables, elect_yr_tocs)

******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
********

* In 2016, year summary is index 10 no shape conflict
* In 2018, year summary tables are index 10, 11
* In 2020, year summary tables are index 10, 11 but no shape conflict
* In 2022, year summary **is not worth it**

In [ ]:
# for i, table in enumerate(elect_yr_tables['2018']):
#     print(f'Table {i}:')
#     display(table)
#     if i == 15:
#         break

In [ ]:
# for u in election_collection_raw[year][1][0].iloc[:,4]:
#     if isinstance(u, str) and 'elected' in u:
#         print('faslidfjasedfhaedfng')
        
# for u in election_collection_raw[year][1][1].iloc[:,4]:
#     if isinstance(u, str) and 'elected' in u:
#         print('faslidfjasedfhaedfng')

In [ ]:
# print(election_collection_raw[year][1][0].shape)
# print(election_collection_raw[year][1][1].shape)

In [ ]:
# test_list = [election_collection_raw[year][2][8], election_collection_raw[year][2][3]]
# for u in test_list:
    
#     display(u.iloc[:,2])

In [ ]:
# election_collection_raw[year][1][1]
# for i, df in enumerate(election_collection_raw[year][2]):
#     display(df)

## Loading `Election_Collection`

In [37]:
# with open('election_collection_raw.pickle', 'wb') as f:
#     pickle.dump(election_collection_raw, f)
#     f.close()

In [11]:
with open('election_collection_raw.pickle', 'rb') as f:
    election_collection_raw = pickle.load(f)
    f.close()

In [12]:
## Must do this befor running code for state elections. Error in code
do_not_use = election_collection_raw['2018'][2].pop(0)

## Separation/Cleaning

### Testing

In [ ]:
# test_res_dict = yr_sum_formatter(test_res)
# for table in test_res_dict['2008']:
#     display(table)

In [ ]:
# year = '2006'

# display(election_collection_raw.keys())
# print('\n')
# for grp in election_collection_raw[year]:
#     print(len(grp))
# print('\n')
# display(election_collection_raw[year][0])
# display(election_collection_raw[year][1])
# display(election_collection_raw[year][2])

In [ ]:
# test_list = []
# test_list.append(election_collection_raw['1918'])
# test_list.append(election_collection_raw['1920'])
# test_list.append(election_collection_raw['1976'])
# test_list.append(election_collection_raw['2006'])
# test_list.append(election_collection_raw['2008'])

# test_list = ['1918', '1920', '1976', '2006', '2008']
# test_dict = {}
# for year in test_list:
#     test_dict[year] = election_collection_raw[year]
# test_dict['2008']

In [ ]:
# type(election_collection_raw['1994'])

In [ ]:
# print(type(election_collection_raw['1920']))
# print(type(election_collection_raw['1976']))
# type(test_res['1920'])

In [ ]:
# test_res = yr_summary_collector(test_dict)
# for t in test_res:
#     display(test_res[t])

In [2]:
# def gen_election_cleaner(df, verbose=1):
#     import pandas as pd
    
#     df_cln = df.copy()
    
#     if verbose in [1,2]:
#         ## Printing startpoint
#         print('O.G. Copy:')
#         print('---'*20)
#         display(df_cln)

#     ## Setting columns manually + Q.C.
#     df_cln.columns = ['Null', 'Party', 'Candidate', 'Votes', '%of_Vote']
#     if verbose == 2:
#         print('\nNew Column Headers:')
#         print('---'*20)
#         display(df_cln)

#     ## Dropping irrelevant information + Q.C.
#     df_cln.drop(index=[0, 6, 8], inplace=True)
#     if verbose == 2:
#         print('\nDropping Rows:')
#         print('---'*20)
#         display(df_cln)

#     ## Flip and place Turnout as its own feature + Q.C.
#     for idx, v, k in zip(df_cln.index, df_cln['Null'], df_cln['Null'].isna()):
#         if not k:
#             df_cln[v] = df_cln['Party'][idx]

#     ## Removing the row with turnout info/column with nulls + Q.C.
#     df_cln.drop(columns='Null', index=7, inplace=True)
#     if verbose in [1,2]:
#         print('\nFinal Version:')
#         print('---'*20)
#         display(df_cln)

#     return df_cln

### Working..

**To map the tables to states I need to:**
* Loop through `election_collection_raw`
* Grab each section of state summaries
* Loop through each section
* Clean each state summary
* Place into container
* Loop through container
* Map each senator to the state
* Update table

In [ ]:
# year = '1994'

# for i, table in enumerate(election_collection_raw[year][2]):
#     print(i)
#     display(table)

# ## Confirmed idx 7, 9, 14 yr 1994
# test_5 = election_collection_raw[year][2][4]
# test_5

# # pd.DataFrame.drop()
# print(f'DF shape: {test_5.shape}')
# st_election_formatter(test_5)

# yr_st_dict['1994']
# yr_st_elects['2018'][0]

# display(yr_sum_dict['2020'])

> **Need to add special code for years past 2018 to account for 'election dates' table.**

In [13]:
yr_st_elects = st_election_collector(election_collection_raw)
print(len(yr_st_elects))
yr_st_dict = st_election_cleaner(yr_st_elects)

53


In [81]:
yr_sum_tables = yr_summary_collector(election_collection_raw)
yr_sum_dict = yr_sum_formatter(yr_sum_tables)
name_lookup_df = master_tabler(yr_sum_dict)

3

3

3

3

3

...

3

3

3

2

3

In [85]:
# display(name_lookup_df['Year'].unique())
print(len(name_lookup_df['Year'].unique()))
print(name_lookup_df.shape)
name_lookup_df.head(10)

53
(1899, 3)


,State_id,Incumbent,Year
0,Alabama,Francis S. White,1914
1,Arizona,Marcus A. Smith,1914
2,Arkansas,James Paul Clarke,1914
3,California,George Clement Perkins,1914
4,Colorado,Charles S. Thomas,1914
5,Connecticut,Frank B. Brandegee,1914
6,Florida,Duncan U. Fletcher,1914
7,Georgia,M. Hoke Smith,1914
8,Idaho,James H. Brady,1914
9,Illinois,Lawrence Y. Sherman,1914


In [112]:
test = list(name_lookup_df['Incumbent'].head(10))


if 'James Paul Clarke' in test:
    tester = name_lookup_df[name_lookup_df['Incumbent'] == 'James Paul Clarke']['State_id']
    print(type(tester))
tester.values

for i in tester:
    if not '(' in i:
        print(i)

<class 'pandas.core.series.Series'>
Arkansas


STARTER CODE FOR MAPPING FUNCTION

In [116]:
year = '2006'

for table in yr_st_dict[year]:
    table['Incumb_Y'] = 0
    for i, name in enumerate(table['Candidate']):
        if '(Incumbent)' in name:
            name = name.replace(' (Incumbent)', '')
            ## Add counter for incumbent or not
            table['Incumb_Y'].iloc[i] = 1
        if name in list(name_lookup_df['Incumbent']):
            state = name_lookup_df[name_lookup_df['Incumbent'] == name].copy()
            state_n = state['State_id'].values
            for st in state_n:
                if not '(' in st:
                    table['State'] = st
    display(table)

C:\Users\d_ful\Anaconda3\envs\learn-env\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Party,Candidate,Votes,%,Total votes,Incumb_Y,State
1,Republican,Jon Kyl (Incumbent),297636,99.5%,297791,1,Arizona
2,Republican,Write-ins,155,0.05%,297791,0,Arizona


,Party,Candidate,Votes,%,Total votes,Incumb_Y
1,Democratic,Jim Pederson,214455,100.00%,214455,0


,Party,Candidate,Votes,%,Total votes,Incumb_Y
1,Libertarian,Richard Mack,3311,100.00%,3311,0


,Party,Candidate,Votes,%,±,Majority,Turnout,Incumb_Y,State
1,Republican,Jon Kyl (Incumbent),814398,53.34%,-25.98%,150257,1526782,1,Arizona
2,Democratic,Jim Pederson,664141,43.50%,+43.50%,150257,1526782,0,Arizona
3,Libertarian,Richard Mack,48231,3.16%,-1.90%,150257,1526782,0,Arizona


...

,Party,Candidate,Votes,%,Incumb_Y
1,Republican,Robert Lorge,194633,99.73%,0
2,Republican Party,Other,530,0.27%,0


,Party,Candidate,Votes,%,±,Majority,Turnout,Incumb_Y,State
1,Democratic,Herb Kohl (Incumbent),1439214,67.31%,+5.8%,808915,2138297,1,Wisconsin
2,Republican,Robert Lorge,630299,29.48%,-7.5%,808915,2138297,0,Wisconsin


,Party,Candidate,Votes,%,Total votes,Incumb_Y
1,Democratic,Dale Groutage,24924,100.00%,24924,0


,Party,Candidate,Votes,%,Total votes,Incumb_Y
1,Republican,Craig Thomas (Incumbent),78211,100.00%,78211,1


,Party,Candidate,Votes,%,±,Majority,Turnout,Incumb_Y,State
1,Republican,Craig L. Thomas (Incumbent),135174,69.99%,-3.78%,77503,193136,1,Wyoming
2,Democratic,Dale Groutage,57671,29.86%,+7.82%,77503,193136,0,Wyoming


In [84]:
for table in yr_st_dict['2006']:
    display(table)

,Party,Candidate,Votes,%,Total votes
1,Republican,Jon Kyl (Incumbent),297636,99.5%,297791
2,Republican,Write-ins,155,0.05%,297791


,Party,Candidate,Votes,%,Total votes
1,Democratic,Jim Pederson,214455,100.00%,214455


,Party,Candidate,Votes,%,Total votes
1,Libertarian,Richard Mack,3311,100.00%,3311


,Party,Candidate,Votes,%,±,Majority,Turnout
1,Republican,Jon Kyl (Incumbent),814398,53.34%,-25.98%,150257,1526782
2,Democratic,Jim Pederson,664141,43.50%,+43.50%,150257,1526782
3,Libertarian,Richard Mack,48231,3.16%,-1.90%,150257,1526782


,Party,Candidate,Votes,%
1,Republican,Dick Mountjoy,1560472,100.00%
2,American Independent,Don J. Grundmann,30787,100.00%
3,Libertarian,Michael S. Metti,16742,100.00%
4,Peace and Freedom,Marsha Feinland,4109,100.00%


...

,Party,Candidate,Votes,%
1,Republican,Robert Lorge,194633,99.73%
2,Republican Party,Other,530,0.27%


,Party,Candidate,Votes,%,±,Majority,Turnout
1,Democratic,Herb Kohl (Incumbent),1439214,67.31%,+5.8%,808915,2138297
2,Republican,Robert Lorge,630299,29.48%,-7.5%,808915,2138297


,Party,Candidate,Votes,%,Total votes
1,Democratic,Dale Groutage,24924,100.00%,24924


,Party,Candidate,Votes,%,Total votes
1,Republican,Craig Thomas (Incumbent),78211,100.00%,78211


,Party,Candidate,Votes,%,±,Majority,Turnout
1,Republican,Craig L. Thomas (Incumbent),135174,69.99%,-3.78%,77503,193136
2,Democratic,Dale Groutage,57671,29.86%,+7.82%,77503,193136


In [43]:
yr_sum_tables['2018']

,0,1,2,3,4,5
0,State(linked tosummaries below),Incumbent,Results,Candidates,NaN,NaN
1,Senator,Party,Electoralhistory,NaN,NaN,NaN
2,Minnesota(Class 2),Tina Smith,Democratic,2018 (Appointed),Interim appointee elected.,Tina Smith (Democratic) 53.0% Karin Housley (R...
3,Mississippi(Class 2),Cindy Hyde-Smith,Republican,2018 (Appointed),Interim appointee elected.,Cindy Hyde-Smith (Republican) 53.6%[27] Mike E...


In [46]:
for year in yr_sum_dict:
    coll = yr_sum_dict[year]
    for tab in coll:
        display(tab)

,State,Senator,Party,Electoral_History,Results,Candidates
0,Alabama,Francis S. White,Democratic,1914 (Special),Incumbent retired.New senator elected.Democrat...,Oscar Underwood (Democratic) 90.2% Alex Birch ...
1,Arizona,Marcus A. Smith,Democratic,1912,Incumbent re-elected.,Marcus A. Smith (Democratic) 53.23% Don Lorenz...
2,Arkansas,James Paul Clarke,Democratic,19031909,Incumbent re-elected.,James Paul Clarke (Democratic) 74.88% Harry H....
3,California,George Clement Perkins,Republican,1893 (Appointed)1895 (Special)189719031909,Incumbent retired.New senator elected.Democrat...,James D. Phelan (Democratic) 31.59% Francis J....
4,Colorado,Charles S. Thomas,Democratic,1913 (Special),Incumbent re-elected.,Charles S. Thomas (Democratic) 40.3% Hubert Wo...
5,Connecticut,Frank B. Brandegee,Republican,1905 (Special)1909,Incumbent re-elected.,Frank B. Brandegee (Republican) 49.77% Simeon ...
6,Florida,Duncan U. Fletcher,Democratic,1909 (Appointed)1909 (Special),Incumbent re-elected.,Duncan U. Fletcher (Democratic) Unopposed[9]
7,Georgia,M. Hoke Smith,Democratic,1911 (Special).,Incumbent re-elected.,M. Hoke Smith (Democratic) 68.48% C. W. McClur...
8,Idaho,James H. Brady,Republican,1913 (Special),Incumbent re-elected.,James H. Brady (Republican) 43.89% James H. Ha...
9,Illinois,Lawrence Y. Sherman,Republican,1913 (Special),Incumbent re-elected.,Lawrence Sherman (Republican) 38.46% Roger Cha...


,State_id,Incumbent,Year
0,Alabama,Francis S. White,1914
1,Arizona,Marcus A. Smith,1914
2,Arkansas,James Paul Clarke,1914
3,California,George Clement Perkins,1914
4,Colorado,Charles S. Thomas,1914
5,Connecticut,Frank B. Brandegee,1914
6,Florida,Duncan U. Fletcher,1914
7,Georgia,M. Hoke Smith,1914
8,Idaho,James H. Brady,1914
9,Illinois,Lawrence Y. Sherman,1914


,State,Senator,Party,Electoral_History,Results,Candidates
0,Arizona,Henry F. Ashurst,Democratic,1912,Incumbent re-elected.,"Henry F. Ashurst (Democratic), 55.36% Joseph H..."
1,California,John D. Works,Republican,1911,Incumbent retired.New senator elected.[5]Repub...,"Hiram Johnson (Republican), 61.09% George S. P..."
2,Connecticut,George P. McLean,Republican,1911,Incumbent re-elected.[6],"George P. McLean (Republican), 50.17% Homer St..."
3,Delaware,Henry A. du Pont,Republican,1906 (Special)1911,Incumbent lost re-election.New senator elected...,"Josiah O. Wolcott (Democratic), 49.67% Henry A..."
4,Florida,Nathan P. Bryan,Democratic,1911 (Appointed)1911 (Special),Incumbent lost renomination.New senator electe...,"Park Trammell (Democratic), 82.86% William O'N..."
5,Indiana,John W. Kern,Democratic,1911,Incumbent lost re-election.New senator elected...,"Harry Stewart New (Republican), 47.77% John W...."
6,Maine,Charles Fletcher Johnson,Democratic,1910,Incumbent lost re-election.New senator elected...,"Frederick Hale (Republican), 52.72% Charles Fl..."
7,Maryland,Blair Lee I,Democratic,1913 (Special),Incumbent lost renomination.New senator electe...,"Joseph I. France (Republican), 49.32% David Jo..."
8,Massachusetts,Henry Cabot Lodge,Republican,1893189919051911,Incumbent re-elected.[13],"Henry Cabot Lodge (Republican), 51.68% John F...."
9,Michigan,Charles E. Townsend,Republican,1910,Incumbent re-elected.[13],"Charles E. Townsend (Republican), 56.34% Lawre..."


,State_id,Incumbent,Year
0,Arizona,Henry F. Ashurst,1916
1,California,John D. Works,1916
2,Connecticut,George P. McLean,1916
3,Delaware,Henry A. du Pont,1916
4,Florida,Nathan P. Bryan,1916
5,Indiana,John W. Kern,1916
6,Maine,Charles Fletcher Johnson,1916
7,Maryland,Blair Lee I,1916
8,Massachusetts,Henry Cabot Lodge,1916
9,Michigan,Charles E. Townsend,1916


,State,Senator,Party,Electoral_History,Results,Candidates
0,Alabama,John H. Bankhead,Democratic,1907 (Appointed)1907 (Special)1911 (Early),Incumbent re-elected.,John H. Bankhead (Democratic) Unopposed[8]
1,Arkansas,Joseph Robinson,Democratic,1913,Incumbent re-elected.,Joseph Robinson (Democratic) Unopposed[9]
2,Colorado,John F. Shafroth,Democratic,1913,Incumbent lost re-election.New senator elected...,"Lawrence C. Phipps (Republican), 49.49% John F..."
3,Delaware,"Willard Saulsbury, Jr.",Democratic,1913,Incumbent lost re-election.New senator elected...,"L. Heisler Ball (Republican), 51.17% Willard S..."
4,Georgia,Thomas W. Hardwick,Democratic,1914 (Special),Incumbent lost renomination.New senator electe...,"William J. Harris (Democratic), 88.34% G. H. W..."
5,Idaho,William Borah,Republican,19071913,Incumbent re-elected.,"William Borah (Republican), 67.21% Frank L. Mo..."
6,Illinois,J. Hamilton Lewis,Democratic,1913 (Late),Incumbent lost re-election.New senator elected...,"Joseph M. McCormick (Republican), 50.5% J. Ham..."
7,Iowa,William S. Kenyon,Republican,1911 (Special)1913,Incumbent re-elected.,"William S. Kenyon (Republican), 65.4% Charles ..."
8,Kansas,William Thompson,Democratic,1913,Incumbent lost re-election.New senator elected...,"Arthur Capper (Republican), 63.69% William Tho..."
9,Kentucky,George B. Martin,Democratic,1918 (Appointed),Incumbent retired.New senator elected.Democrat...,"Augustus O. Stanley (Democratic), 50.77% Ben B..."


...

,State_id,Incumbent,Year
0,Arizona,Jon Kyl,2012
1,California,Dianne Feinstein,2012
2,Connecticut,Joe Lieberman,2012
3,Delaware,Tom Carper,2012
4,Florida,Bill Nelson,2012
5,Hawaii,Daniel Akaka,2012
6,Indiana,Richard Lugar,2012
7,Maine,Olympia Snowe,2012
8,Maryland,Ben Cardin,2012
9,Massachusetts,Scott Brown,2012


,State,Senator,Party,Electoral_History,Results,Candidates
0,Alabama,Richard Shelby,Republican,19861992199820042010,Incumbent re-elected.,Richard Shelby (Republican)[16] 64.0% Ron Crum...
1,Alaska,Lisa Murkowski,Republican,2002 (Appointed)20042010,Incumbent re-elected.,Lisa Murkowski (Republican)[18] 44% Joe Miller...
2,Arizona,John McCain,Republican,19861992199820042010,Incumbent re-elected.,John McCain (Republican)[23] 53% Ann Kirkpatri...
3,Arkansas,John Boozman,Republican,2010,Incumbent re-elected.,John Boozman (Republican)[28] 59.7% Conner Eld...
4,California,Barbara Boxer,Democratic,1992199820042010,Incumbent retired.New senator elected.Democrat...,Kamala Harris (Democratic)[31] 61.8% Loretta S...
5,Colorado,Michael Bennet,Democratic,2009 (Appointed)2010,Incumbent re-elected.,Michael Bennet (Democratic)[33] 50.0% Darryl G...
6,Connecticut,Richard Blumenthal,Democratic,2010,Incumbent re-elected.,Richard Blumenthal (Democratic)[37] 63.2% Dan ...
7,Florida,Marco Rubio,Republican,2010,Incumbent re-elected.,Marco Rubio (Republican)[41] 52.0% Patrick Mur...
8,Georgia,Johnny Isakson,Republican,20042010,Incumbent re-elected.,Johnny Isakson (Republican)[44] 54.8% Jim Bark...
9,Hawaii,Brian Schatz,Democratic,2012 (Appointed)2014 (Special),Incumbent re-elected.,Brian Schatz (Democratic)[47] 73.6% John Carro...


,State_id,Incumbent,Year
0,Alabama,Richard Shelby,2016
1,Alaska,Lisa Murkowski,2016
2,Arizona,John McCain,2016
3,Arkansas,John Boozman,2016
4,California,Barbara Boxer,2016
5,Colorado,Michael Bennet,2016
6,Connecticut,Richard Blumenthal,2016
7,Florida,Marco Rubio,2016
8,Georgia,Johnny Isakson,2016
9,Hawaii,Brian Schatz,2016


,State,Senator,Party,Electoral_History,Results,Candidates
0,Minnesota(Class_2),Tina Smith,Democratic,2018 (Appointed),Interim appointee elected.,Tina Smith (Democratic) 53.0% Karin Housley (R...
1,Mississippi(Class_2),Cindy Hyde-Smith,Republican,2018 (Appointed),Interim appointee elected.,Cindy Hyde-Smith (Republican) 53.6%[27] Mike E...


,State_id,Incumbent,Year
0,Minnesota(Class_2),Tina Smith,2018
1,Mississippi(Class_2),Cindy Hyde-Smith,2018


In [20]:
count = 0
test_list = []
for table in election_collection_raw[year][2]:
    try:
        res = gen_election_cleaner(table, verbose=0)
        test_list.append(res)
        count += 1
    except:
        continue
        
    if count == 3:
        break

In [22]:
for t in test_list:
    display(t)

,Party,Candidate,Votes,%of_Vote,Total votes,Turnout
1,Democratic,Dianne Feinstein (Incumbent),3979152,46.74%,8636900,NaN
2,Republican,Michael Huffington,3817025,44.83%,8636900,NaN
3,Peace and Freedom,Elizabeth Cervantes Barron,255301,3.00%,8636900,NaN
4,Libertarian,Richard Benjamin Boddie,179100,2.10%,8636900,NaN
5,American Independent,Paul Meeuwenberg,142771,1.68%,8636900,NaN
9,8636900,100.00%,NaN,NaN,8636900,NaN
10,NaN,46.98%,NaN,NaN,8636900,NaN
11,Democratic hold,NaN,NaN,NaN,8636900,NaN


,Party,Candidate,Votes,%of_Vote,Total votes
1,Republican,Spencer Abraham,1578770,51.9%,3022869
2,Democratic,Bob Carr,1300960,42.8%,3022869
3,Libertarian,Jon Coon,128393,4.2%,3022869
4,Workers World,William Roundtree,20010,0.7%,3022869
5,Natural Law,Chris Wege,14746,0.5%,3022869


,Party,Candidate,Votes,%of_Vote,Majority
1,Republican,Rod Grams,869653,49.1%,87793
2,Democratic,Ann Wynia,781860,44.1%,87793
3,Reform,Dean Barkley,95400,5.4%,87793
4,Independent,Candice E. Sjostrom,15920,0.9%,87793
5,Natural Law Party,Stephen Johnson,5054,0.3%,87793
9,Republican hold,NaN,NaN,NaN,87793


**To format the tables I need to:**
* For each table in the dictionary, set the columns to `'State', 'Senator', 'Party', 'Electoral_History', 'Results', 'Candidates'`.
* If any of the rows look like row 0 & 1, remove them.
* Store the formatted tables back into container

In [31]:
# test_df['Incumbent'].duplicated(keep=False)
test_df.groupby('Incumbent').indices

{'Albert B. Cummins': array([ 11, 122], dtype=int64),
 'Albert B. Fall': array([90], dtype=int64),
 'Alva B. Adams': array([214], dtype=int64),
 'Andrieus A. Jones': array([162], dtype=int64),
 'Arthur Capper': array([190], dtype=int64),
 'Asle Gronna': array([21], dtype=int64),
 'Asle J. Gronna': array([132], dtype=int64),
 'Atlee Pomerene': array([ 55, 165], dtype=int64),
 'Augustus O. Stanley': array([191], dtype=int64),
 'Benjamin F. Shively': array([10], dtype=int64),
 'Bert M. Fernald': array([ 81, 193], dtype=int64),
 'Blair Lee I': array([42], dtype=int64),
 'Boies Penrose': array([ 25, 136], dtype=int64),
 'Braxton B. Comer': array([143], dtype=int64),
 'Caroll S. Page': array([171], dtype=int64),
 'Carroll S. Page': array([61], dtype=int64),
 'Carter Glass': array([144, 211], dtype=int64),
 'Charles A. Culberson': array([169], dtype=int64),
 'Charles A. Rawson': array([179], dtype=int64),
 'Charles Allen Culberson': array([59], dtype=int64),
 'Charles B. Henderson': array([12

* Create a table with names as index and state as values .loc name of candidates to get state value
* Use table of each year's general summary to then pull the general elections out of whole list (index 2 in colleciton) based upon names in table + non-uniform party column
    * Need to merge (if special election) tables in collection index 1

In [20]:
test = {'1918': ['fire'], '1920': 'jet', '1976': 'hello'}
test['1918'].append('bromine')
for i in test:
    print(i)
print(test['1918'])

1918
1920
1976
['fire', 'bromine']


* 1924 = 10
* 1926 = 12
* 1928 = 10
* Pull only tables with `shape[1]` of 5 or 6.
* Table 2 is always summary of senate leaders

Why after the 17th amendment?? [Link](https://en.wikipedia.org/wiki/Seventeenth_Amendment_to_the_United_States_Constitution)

For 1920 forward, proceed like 1976. Prior to this, will need to use first table with general info.

* **Clean up NaN value table from top of pages** -- Use index 2 to grab  nested table from `'infobox vevent'`.
* **Take list of states of top tables** and map to dfs
* Get demographic info wikidata
* eda time series of state party (heatmap)
* NLP on names, etc.
* [link for slider viz](https://medium.com/@sjacks/the-journey-to-an-attractive-visualization-bac019506a49)